In [3]:
from pathlib import Path
import shutil
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image as PILImage
import re

WORK = Path(os.environ.get("WORK", Path.cwd()))
PROJECT_ROOT = WORK / "color-concept-entanglement"

DATA = PROJECT_ROOT / "data"
IMAGE_BASE_COLOR = DATA / "color_images"
IMAGE_BASE_SHAPES = DATA / "shapes" / "shape_colored"

EXPORT_ROOT = PROJECT_ROOT / "hf_dataset_export"
EXPORT_ROOT.mkdir(exist_ok=True)

EXPORT_IMAGES = EXPORT_ROOT / "images"

In [2]:
# Load the datasets that where also used in the human evaluation
prior_df = pd.read_csv(DATA / "prolific_stimuli" / f"stimulus_table_image_priors_prolific.csv")
counterfact_df = pd.read_csv(DATA / "prolific_stimuli" / f"stimulus_table_counterfact_prolific.csv")
shape_df = pd.read_csv(DATA / "prolific_stimuli" / f"stimulus_table_shapes_prolific.csv")

display(prior_df.head(), prior_df.shape, counterfact_df.head(), counterfact_df.shape, shape_df.head(), shape_df.shape)

,image_path,object,stimulus_type,manipulation_color,target_color,variant_region,percent_colored,variant_label,mode
0,color_images/gpt-4o/image_priors/cheese_1_78f6...,cheese,correct_prior,yellow,white,BG,80,BG 80% (seq),seq
1,color_images/gpt-4o/image_priors/espresso_make...,espresso maker,correct_prior,red,red,FG,5,FG 5% (seq),seq
2,color_images/gpt-4o/image_priors/tile_roof_2_f...,tile roof,correct_prior,red,red,FG,100,FG 100% (seq),seq
3,color_images/gpt-4o/image_priors/cloud_3_29898...,cloud,correct_prior,grey,grey,FG,55,FG 55% (seq),seq
4,color_images/gpt-4o/image_priors/frilled_lizar...,frilled lizard,correct_prior,brown,brown,FG,100,FG 100% (seq),seq


(1260, 9)

,image_path,object,stimulus_type,manipulation_color,target_color,variant_region,percent_colored,variant_label,mode
0,color_images/gpt-4o/counterfact/rose_3_6471302...,rose,counterfact,blue,blue,FG,100,FG 100% (seq),seq
1,color_images/gpt-4o/counterfact/Sealyham_terri...,sealyham terrier,counterfact,purple,purple,FG,60,FG 60% (seq),seq
2,color_images/gpt-4o/counterfact/iguana_2_a2663...,iguana,counterfact,orange,orange,FG,10,FG 10% (seq),seq
3,color_images/gpt-4o/counterfact/hartebeest_3_5...,hartebeest,counterfact,red,red,FG,55,FG 55% (seq),seq
4,color_images/gpt-4o/counterfact/mouse_2_cf4ddb...,mouse,counterfact,red,red,FG,20,FG 20% (seq),seq


(412, 9)

,image_path,object,stimulus_type,manipulation_color,target_color,variant_region,percent_colored,variant_label,mode
0,shapes/shape_colored/hexagon_v3_yellow/FG_060_...,hexagon,shape,yellow,yellow,FG,60,FG 60% (seq),seq
1,shapes/shape_colored/pentagon_v0_purple/FG_055...,pentagon,shape,purple,purple,FG,55,FG 55% (seq),seq
2,shapes/shape_colored/square_v3_blue/FG_005_seq...,square,shape,blue,blue,FG,5,FG 5% (seq),seq
3,shapes/shape_colored/triangle_v0_brown/FG_050_...,triangle,shape,brown,brown,FG,50,FG 50% (seq),seq
4,shapes/shape_colored/pentagon_v2_orange/FG_090...,pentagon,shape,orange,orange,FG,90,FG 90% (seq),seq


(1331, 9)

In [4]:
COLOR_BASE = DATA / "color_images"
SHAPE_BASE = DATA / "shapes" / "shape_colored"


def clean_stimulus_name(folder_name):
    # remove hash and resized parts
    folder_name = re.sub(r"_\d+_[a-f0-9]+_resized", "", folder_name)
    folder_name = folder_name.replace("_resized", "")
    return folder_name


def clean_image_name(stimulus_name, filename):
    # remove _seq
    filename = filename.replace("_seq", "")
    return f"{stimulus_name}_{filename}"


def resolve_original_path(p):
    p = Path(p)

    if p.is_absolute():
        return p

    if str(p).startswith("color_images"):
        return DATA / p
    elif str(p).startswith("shapes"):
        return DATA / p
    else:
        raise ValueError(f"Cannot resolve original path: {p}")


def make_relative_export_path(original_path, subfolder):
    original_path = Path(original_path)

    stimulus_dir = original_path.parent.name
    filename = original_path.name

    return Path("images") / subfolder / stimulus_dir / filename



def export_dataset(df, filename, subfolder):
    df_export = df.copy()

    original_paths = []
    export_paths = []

    for p in df_export["image_path"]:
        orig = resolve_original_path(p)

        original_folder = orig.parent.name
        clean_folder = clean_stimulus_name(original_folder)

        original_filename = orig.name
        clean_filename = clean_image_name(clean_folder, original_filename)

        exp = Path("images") / subfolder / clean_folder / clean_filename

        original_paths.append(orig)
        export_paths.append(exp)


    df_export["image_path"] = [str(p) for p in export_paths]

    unique_pairs = dict(zip(export_paths, original_paths))

    for rel_path, orig_path in tqdm(unique_pairs.items(),
                                    desc=f"Copying images for {filename}"):

        destination = EXPORT_ROOT / rel_path
        destination.parent.mkdir(parents=True, exist_ok=True)

        if not destination.exists():
            shutil.copy2(orig_path, destination)

    df_export.to_csv(EXPORT_ROOT / filename, index=False)

    print(f"{filename}: copied {len(unique_pairs)} images")

    return df_export

In [5]:
prior_export = export_dataset(prior_df, subfolder="color_priors", filename="color_priors.csv")

Copying images for color_priors.csv:   0%|          | 0/1260 [00:00<?, ?it/s]

Copying images for color_priors.csv: 100%|██████████| 1260/1260 [00:07<00:00, 161.78it/s]

color_priors.csv: copied 1260 images


In [6]:
counterfact_export = export_dataset(counterfact_df, subfolder="counterfact", filename="counterfact.csv")

Copying images for counterfact.csv: 100%|██████████| 412/412 [00:02<00:00, 154.38it/s]

counterfact.csv: copied 412 images


In [7]:
shape_export = export_dataset(shape_df, subfolder="shapes", filename="shapes.csv")

Copying images for shapes.csv: 100%|██████████| 1331/1331 [00:08<00:00, 155.23it/s]

shapes.csv: copied 1331 images


In [8]:
def validate_export(df_export, split_name):
    
    print(f"\n Validating split: {split_name}")
    
    total_rows = len(df_export)
    unique_paths = df_export["image_path"].unique()
    
    print(f"Total rows: {total_rows}")
    print(f"Unique image paths: {len(unique_paths)}")
    
    missing_files = []
    wrong_size = []
    
    for rel_path in tqdm(unique_paths, desc=f"Checking images ({split_name})"):
        
        full_path = EXPORT_ROOT / rel_path
        
        # Check file exists
        if not full_path.exists():
            missing_files.append(rel_path)
            continue
        
        # Check image size
        try:
            with PILImage.open(full_path) as img:
                if img.size != (512, 512):
                    wrong_size.append((rel_path, img.size))
        except Exception as e:
            print(f"Could not open {rel_path}: {e}")
            missing_files.append(rel_path)
    
    # Report results
    if not missing_files:
        print("All image files exist.")
    else:
        print(f"Missing files: {len(missing_files)}")
        print(missing_files[:5])
    
    if not wrong_size:
        print("All images are 512x512.")
    else:
        print(f"Images with wrong size: {len(wrong_size)}")
        print(wrong_size[:5])
    
    print("Validation complete.\n")


In [9]:
validate_export(prior_export, "color_priors")
validate_export(counterfact_export, "counterfact")
validate_export(shape_export, "shapes")


 Validating split: color_priors
Total rows: 1260
Unique image paths: 1260


Checking images (color_priors): 100%|██████████| 1260/1260 [00:00<00:00, 2062.97it/s]


All image files exist.
All images are 512x512.
Validation complete.


 Validating split: counterfact
Total rows: 412
Unique image paths: 412


Checking images (counterfact): 100%|██████████| 412/412 [00:00<00:00, 2768.11it/s]


All image files exist.
All images are 512x512.
Validation complete.


 Validating split: shapes
Total rows: 1331
Unique image paths: 1331


Checking images (shapes): 100%|██████████| 1331/1331 [00:00<00:00, 2943.42it/s]

All image files exist.
All images are 512x512.
Validation complete.

